In [ ]:
# Imports
import numpy as np
from scipy.special import erf
from scipy.optimize import curve_fit, least_squares
from astropy.io import fits
import matplotlib.pyplot as pl
from scipy import ndimage
from scipy.interpolate import interp1d
from astropy.table import Table
import matplotlib.colors as colors
from scipy.special import wofz
%matplotlib inline

In [ ]:
def gaussbin(x, amp, cen, sig, const=0, dx=1.0):
    """1-D gaussian with pixel binning
    
    This function returns a binned Gaussian
    par = [height, center, sigma]
    
    Parameters
    ----------
    x : array
       The array of X-values.
    amp : float
       The Gaussian height/amplitude.
    cen : float
       The central position of the Gaussian.
    sig : float
       The Gaussian sigma.
    const : float, optional, default=0.0
       A constant offset.
    dx : float, optional, default=1.0
      The width of each "pixel" (scalar).
    
    Returns
    -------
    geval : array
          The binned Gaussian in the pixel

    """

    xcen = np.array(x)-cen             # relative to the center
    x1cen = xcen - 0.5*dx  # left side of bin
    x2cen = xcen + 0.5*dx  # right side of bin

    t1cen = x1cen/(np.sqrt(2.0)*sig)  # scale to a unitless Gaussian
    t2cen = x2cen/(np.sqrt(2.0)*sig)

    # For each value we need to calculate two integrals
    #  one on the left side and one on the right side

    # Evaluate each point
    #   ERF = 2/sqrt(pi) * Integral(t=0-z) exp(-t^2) dt
    #   negative for negative z
    geval_lower = erf(t1cen)
    geval_upper = erf(t2cen)

    geval = amp*np.sqrt(2.0)*sig * np.sqrt(np.pi)/2.0 * ( geval_upper - geval_lower )
    geval += const   # add constant offset

    return geval


In [ ]:
def gaussian(x, amp, cen, sig, const=0):
    """1-D gaussian: gaussian(x, amp, cen, sig)"""
    return amp * np.exp(-(x-cen)**2 / (2*sig**2)) + const

In [ ]:
def gaussfit(x,y,initpar=None,sigma=None,bounds=(-np.inf,np.inf),binned=False):
    """Fit a Gaussian to data."""
    if initpar is None:
        initpar = [np.max(y),x[np.argmax(y)],1.0,np.median(y)]
    func = gaussian
    if binned is True: func=gaussbin
    return curve_fit(func, x, y, p0=initpar, sigma=sigma, bounds=bounds)

In [ ]:
# Load the object spectrum 2D image
im,head = fits.getdata('HD12707_comb.fits',header=True)
imerr = np.sqrt(np.maximum(im,1)/1.42)   # gain=1.420 e-/ADU

In [ ]:
# Load the comparison/arc lamp spectrum 2D image
cim,chead = fits.getdata('HD12707_compar1.fits',header=True)
cimerr = np.sqrt(np.maximum(cim,1)/1.42)   # gain=1.420 e-/ADU

In [ ]:
fig = plt.figure(figsize=(12,10))
plt.imshow(im,origin='lower',aspect='equal')

In [ ]:
fig = plt.figure(figsize=(12,10))
plt.imshow(im,origin='lower',aspect='auto')
plt.ylim(150,180)

In [ ]:
fig = plt.figure(figsize=(12,10))
plt.imshow(cim,origin='lower',aspect='equal')

# Trace the Spectrum

In [ ]:
ny,nx = im.shape
x = np.arange(nx)
y = np.arange(ny)

In [ ]:
par,cov=gaussfit(y,im[:,400])
gmodel=gaussian(y,*par)

In [ ]:
plt.plot(y,im[:,400],linewidth=2)
plt.plot(y,gmodel,linewidth=1,linestyle='dashed')

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.plot(y,im[:,400],linewidth=3)
plt.plot(y,mod,linewidth=2,linestyle='dashed')
plt.xlim(150,180)

In [ ]:
def trace(im,yestimate=None,yorder=2,sigorder=4,step=10):
    """ Trace the spectrum.  Spectral dimension is assumed to be on the horizontal axis."""
    ny,nx = im.shape
    if yestimate is None:
        ytot = np.sum(im,axis=1)
        yestimate = np.argmax(ytot)
    # Smooth in spectral dimension
    # a uniform (boxcar) filter with a width of 50
    smim = ndimage.uniform_filter1d(im, 50, 1)
    nstep = nx//step
    # Loop over the columns in steps and fit Gaussians
    tcat = np.zeros(nstep,dtype=np.dtype([('x',float),('pars',float,4)]))
    for i in range(nstep):
        pars,cov = gaussfit(y[yestimate-10:yestimate+10],im[yestimate-10:yestimate+10,step*i+step//2])
        tcat['x'][i] = step*i+step//2
        tcat['pars'][i] = pars
    # Fit polynomail to y vs. x and gaussian sigma vs. x
    ypars = np.polyfit(tcat['x'],tcat['pars'][:,1],yorder)
    sigpars = np.polyfit(tcat['x'],tcat['pars'][:,2],sigorder)
    # Model
    mcat = np.zeros(nx,dtype=np.dtype([('x',float),('y',float),('sigma',float)]))
    xx = np.arange(nx)
    mcat['x'] = xx
    mcat['y'] = np.poly1d(ypars)(xx)
    mcat['sigma'] = np.poly1d(sigpars)(xx)
    return tcat, ypars, sigpars,mcat

In [ ]:
tcat,ypars,sigpars,mcat=trace(im)

In [ ]:
fig = plt.figure(figsize=(14,8))
plt.imshow(im,origin='lower',aspect='auto')
plt.ylim(150,180)
plt.scatter(tcat['x'],tcat['pars'][:,1],c='r',marker='+')

In [ ]:
# Fit a polynomial to Y vs. X
fig = plt.figure(figsize=(12,8))
plt.scatter(tcat['x'],tcat['pars'][:,1],marker='+')
plt.xlabel('Column')
plt.ylabel('Row')
plt.title('Row vs. Column')
plt.plot(mcat['x'],mcat['y'],c='r',label='Fit')
plt.legend()
print(ypars)

In [ ]:
# Fit a polynomial to Gaussian Sigma vs. X
fig = plt.figure(figsize=(12,8))
plt.scatter(tcat['x'],tcat['pars'][:,2],marker='+')
plt.xlabel('Column')
plt.ylabel('Gaussian Sigma')
plt.title('Gaussian Sigma vs. Column')
plt.plot(mcat['x'],mcat['sigma'],c='r',label='Fit')
plt.legend()
print(sigpars)

# Extract the Spectrum

In [ ]:
def boxcar(im):
    """ Boxcar extract the spectrum"""
    ny,nx = im.shape
    ytot = np.sum(im,axis=1)
    yest = np.argmax(ytot)
    # Background subtract
    yblo = int(np.maximum(yest-50,0))
    ybhi = int(np.minimum(yest+50,ny))
    med = np.median(im[yblo:ybhi,:],axis=0)
    medim = np.repeat(med,ny).reshape(ny,nx)
    subim = im.copy()-medim
    # Sum up the flux
    ylo = int(np.maximum(yest-20,0))
    yhi = int(np.minimum(yest+20,ny))
    flux = np.sum(subim[ylo:yhi,:],axis=0)
    return flux

In [ ]:
boxflux = boxcar(im)

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.plot(boxflux)

We can improve the extraction by fitting a Gaussian to the profiles but now holding the center and Gaussian sigma fixed.  Only fit the height (i.e. flux) and constant offset (background).  This will allow us to properly weight each pixels.  This will improve the results especially for low S/N spectra.

In [ ]:
def linefit(x,y,initpar,bounds,err=None):
    # Fit Gaussian profile to data with center and sigma fixed.
    # initpar = [height, center, sigma, constant offset]
    cen = initpar[1]
    sigma = initpar[2]
    def gline(x, amp, const=0):
        """1-D gaussian: gaussian(x, amp, cen, sig)"""
        return amp * np.exp(-(x-cen)**2 / (2*sigma**2)) + const
    line_initpar = [initpar[0],initpar[3]]
    lbounds, ubounds = bounds
    line_bounds = ([lbounds[0],lbounds[3]],[ubounds[0],ubounds[3]])
    return curve_fit(gline, x, y, p0=line_initpar, bounds=line_bounds, sigma=err)

In [ ]:
plt.plot(y[155:175],im[155:175,400])

In [ ]:
Table(mcat)[400]

In [ ]:
initpar = [8e5, 164.906, 1.32455, 0.0]
bnds = ( [0,0,0,0], [1e8,0,0,1e4] )
pars0,cov0 = linefit(y[155:175],im[155:175,400],initpar=initpar,bounds=bnds)
pars = [pars[0],164.906, 1.32455, pars0[1]]
print(pars)
lmodel = gaussian(y[155:175],*pars)

In [ ]:
plt.plot(y[155:175],im[155:175,400])
plt.plot(y[155:175],lmodel,linestyle='dashed')

In [ ]:
def extract(im,imerr=None,mcat=None,nobackground=False):
    """ Extract a spectrum"""
    ny,nx = im.shape
    x = np.arange(nx)
    y = np.arange(ny)
    # No trace information input, get it
    if mcat is None:
        tcat,ypars,sigpars,mcat=trace(im)
    # Loop over the columns and get the flux using the trace information
    cat = np.zeros(nx,dtype=np.dtype([('x',int),('pars',float,2),('perr',float,2),
                                      ('flux',float),('fluxerr',float)]))
    for i in range(nx):
        line = im[:,i].flatten()
        if imerr is not None:
            lineerr = imerr[:,i].flatten()
        else:
            lineerr = np.ones(len(line))   # unweighted
        # Fit the constant offset and the height of the Gaussian
        #  fix the central position and sigma
        ycen = mcat['y'][i]
        ysigma = mcat['sigma'][i]
        ht0 = np.maximum(line[int(np.round(ycen))],0.01)
        initpar = [ht0,ycen,ysigma,np.median(line)]
        if nobackground is True:
            initpar = [ht0,ycen,ysigma,0]
        # Only fit the region fight around the peak
        y0 = int(np.maximum(ymid-50,0))
        y1 = int(np.minimum(ymid+50,ny))
        bnds = ([0,ycen-1e-4,ysigma-1e-4,0],[1.5*ht0,ycen,ysigma,1.5*initpar[3]])
        if nobackground is True:
            bnds = ([0,ycen-1e-4,ysigma-1e-4,0],[1.5*ht0,ycen,ysigma,0.1])
        pars,cov = linefit(y[y0:y1],line[y0:y1],initpar=initpar,bounds=bnds,err=lineerr[y0:y1])
        perr = np.sqrt(np.diag(cov))
        # Gaussian area = ht*wid*sqrt(2*pi)
        flux = pars[0]*ysigma*np.sqrt(2*np.pi)
        fluxerr = perr[0]*ysigma*np.sqrt(2*np.pi)
        cat['x'][i] = i
        cat['pars'][i] = pars
        cat['perr'][i] = perr
        cat['flux'][i] = flux
        cat['fluxerr'][i] = fluxerr
    return cat

In [ ]:
tcat,ypars,sigpars,mcat=trace(im)
scat = extract(im,imerr,mcat)

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.plot(scat['flux'])

In [ ]:
Table(scat)

We need to extract the comparison lamp spectrum the same way we did the object spectrum.

In [ ]:
ccat = extract(cim,cimerr,mcat,nobackground=True)

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.plot(ccat['flux'])

# Wavelength Solution

<img src="hear340_520.gif" width=800 height=600 align="left"/>

The wavelength range that I was targeting was 3700 - 5250 Angstroms.

## Step 1.  Pick out the most prominent lines and find an intial wavelength solution

In [ ]:
# The first line around 150 is 3888.6A
fig = plt.figure(figsize=(12,6))
plt.plot(ccat['flux'])
plt.xlim(140,160)

In [ ]:
pars,cov=gaussfit(x[145:160],ccat['flux'][145:160])
print(pars)

In [ ]:
# The second line around 340 is 4158.6A
fig = plt.figure(figsize=(12,6))
plt.plot(ccat['flux'])
plt.xlim(300,400)

In [ ]:
pars,cov=gaussfit(x[330:345],ccat['flux'][330:345])
print(pars)

In [ ]:
# The third line around 550 is 4471.5A
fig = plt.figure(figsize=(12,6))
plt.plot(ccat['flux'])
plt.xlim(500,600)

In [ ]:
pars,cov=gaussfit(x[540:560],ccat['flux'][540:560])
print(pars)

In [ ]:
# The fourth line around 920 is 5015.7
fig = plt.figure(figsize=(12,6))
plt.plot(ccat['flux'])
plt.xlim(900,960)

In [ ]:
pars,cov=gaussfit(x[910:930],ccat['flux'][910:930])
print(pars)

In [ ]:
# Now fit a polynomial to the wavelengths versus column number.
col = [1.53246075e+02,3.37114623e+02,5.49360035e+02,9.16610137e+02]
wave = [3888.6,4158.6,4471.5,5015.7]

In [ ]:
plt.plot(col,wave,'go--')

In [ ]:
wcoef1 = np.polyfit(col,wave,1)
wmodel1 = np.poly1d(wcoef1)(x)
print(wcoef1)

In [ ]:
plt.plot(col,wave,'go--')
plt.plot(wmodel1)

In [ ]:
plt.plot(col,wave-np.poly1d(wcoef1)(col),'go--')
plt.plot([0,len(x)],[0,0])

In [ ]:
wcoef2 = np.polyfit(col,wave,2)
wmodel2 = np.poly1d(wcoef2)(x)
print(wcoef2)

In [ ]:
plt.plot(col,wave-np.poly1d(wcoef2)(col),'go--')
plt.plot([0,len(x)],[0,0])

## Step: Measure all of the lines and get their wavelengths from a linelist

In [ ]:
def emissionlines(spec,thresh=None):
    """Measure the emission lines in an arc lamp spectrum. """
    nx = len(spec)
    x = np.arange(nx)
    
    # Threshold
    if thresh is None:
        thresh = np.min(spec) + (np.max(spec)-np.min(spec))*0.05
    
    # Detect the peaks
    sleft = np.hstack((0,spec[0:-1]))
    sright = np.hstack((spec[1:],0))
    peaks, = np.where((spec>sleft) & (spec>sright) & (spec>thresh))
    npeaks = len(peaks)
    print(str(npeaks)+' peaks found')
    
    # Loop over the peaks and fit them with Gaussians
    gcat = np.zeros(npeaks,dtype=np.dtype([('x0',int),('x',float),('xerr',float),('pars',float,4),('perr',float,4),
                                           ('flux',float),('fluxerr',float)]))
    resid = spec.copy()
    gmodel = np.zeros(nx)
    for i in range(npeaks):
        x0 = peaks[i]
        xlo = np.maximum(x0-6,0)
        xhi = np.minimum(x0+6,nx)
        initpar = [spec[x0],x0,1,0]
        bnds = ([0,x0-3,0.1,0],[1.5*initpar[0],x0+3,10,1e4])
        pars,cov = gaussfit(x[xlo:xhi],spec[xlo:xhi],initpar,bounds=bnds,binned=True)
        perr = np.sqrt(np.diag(cov))
        gmodel1 = gaussian(x[xlo:xhi],*pars)
        gmodel[xlo:xhi] += (gmodel1-pars[3])
        resid[xlo:xhi] -= (gmodel1-pars[3])
        # Gaussian area = ht*wid*sqrt(2*pi)
        flux = pars[0]*ysigma*np.sqrt(2*np.pi)
        fluxerr = perr[0]*ysigma*np.sqrt(2*np.pi)
        gcat['x0'][i] = x0
        gcat['x'][i] = pars[1]
        gcat['xerr'][i] = perr[1]
        gcat['pars'][i] = pars
        gcat['perr'][i] = perr
        gcat['flux'][i] = flux
        gcat['fluxerr'][i] = fluxerr
        #print(str(i+1)+' '+str(x0)+' '+str(pars))
        
    return gcat, gmodel

In [ ]:
ecat,gmodel = emissionlines(cflux)

In [ ]:
plt.plot(cflux)

In [ ]:
plt.plot(gmodel)
plt.ylim(0,1e5)

In [ ]:
len(ecat)

## Load the HeAr linelist

In [ ]:
linelist = Table.read('hear_linelist.txt',format='ascii.commented_header')

In [ ]:
linelist

### Use our initial wavelength solution to get initial wavelengths for all of our lines

In [ ]:
w0 = np.poly1d(wcoef2)(ecat['x'])

In [ ]:
w0

In [ ]:
plt.scatter(np.arange(len(w0)),w0)

In [ ]:
# Match up the lines to the linelist using a matching radius of 0.5A
match = np.zeros(len(ecat),bool)
wdiff = np.zeros(len(ecat),float)
wmatch = np.zeros(len(cat),np.float64)
xmatch = np.zeros(len(cat),float)
for i in range(len(ecat)):
    dist = np.abs(linelist['wavelength']-w0[i])
    mindist = np.min(dist)
    ind = np.argmin(dist)
    if mindist<0.5:
        match[i] = True
        wmatch[i] = linelist['wavelength'][ind]
        wdiff[i] = linelist['wavelength'][ind]-w0[i]
        xmatch[i] = ecat['x'][i]
    #print(str(i+1)+' '+str(mindist))

In [ ]:
len(wdiff)

In [ ]:
wmatch

In [ ]:
ecat = Table(ecat)
ecat['wave0'] = w0
ecat['wave'] = 0.0
gd, = np.where(match == True)
print(len(gd))
ecat['wave'][gd] = wmatch[gd]

In [ ]:
ecat

## Improve the wavelength solution

In [ ]:
plt.plot(ecat['x'][gd],ecat['wave'][gd],'go--')

In [ ]:
fig = plt.figure(figsize=(12,8))
norm = colors.LogNorm()
plt.scatter(ecat['x'][gd],ecat['wave'][gd]-np.poly1d(wcoef2)(ecat['x'][gd]),
            c=ecat['flux'][gd],marker='o',norm=norm)
plt.errorbar(ecat['x'][gd],ecat['wave'][gd]-np.poly1d(wcoef2)(ecat['x'][gd]),yerr=ecat['xerr'][gd],
             fmt='none',ecolor='gray')
plt.plot([0,len(x)],[0,0])
plt.xlabel('X')
plt.ylabel('Residual Wavelength (Angstroms)')
plt.ylim(-0.5,0.5)
plt.colorbar(label='Flux')

In [ ]:
# Select only good lines with small positional errors
gd, = np.where((ecat['wave']>0) & (ecat['xerr']<0.1) & (ecat['flux']>3e4))
print(len(gd))

In [ ]:
fig = plt.figure(figsize=(12,8))
norm = colors.LogNorm()
plt.scatter(ecat['x'][gd],ecat['wave'][gd]-np.poly1d(wcoef2)(ecat['x'][gd]),
            c=ecat['flux'][gd],marker='o',norm=norm)
plt.errorbar(ecat['x'][gd],ecat['wave'][gd]-np.poly1d(wcoef2)(ecat['x'][gd]),yerr=ecat['xerr'][gd],
             fmt='none',ecolor='gray')
plt.plot([0,len(x)],[0,0])
plt.xlabel('X')
plt.ylabel('Residual Wavelength (Angstroms)')
plt.ylim(-0.5,0.5)
plt.colorbar(label='Flux')

In [ ]:
wcoef3 = np.polyfit(ecat['x'][gd],ecat['wave'][gd],3)
wmodel3 = np.poly1d(wcoef3)(x)
print(wcoef3)

In [ ]:
fig = plt.figure(figsize=(12,8))
norm = colors.LogNorm()
plt.scatter(ecat['x'][gd],ecat['wave'][gd]-np.poly1d(wcoef3)(ecat['x'][gd]),
            c=ecat['flux'][gd],marker='o',norm=norm)
plt.errorbar(ecat['x'][gd],ecat['wave'][gd]-np.poly1d(wcoef3)(ecat['x'][gd]),yerr=ecat['xerr'][gd],
             fmt='none',ecolor='gray')
plt.plot([0,len(x)],[0,0])
plt.xlabel('X')
plt.ylabel('Residual Wavelength (Angstroms)')
plt.ylim(-0.5,0.5)
plt.colorbar(label='Flux')

## Add the final wavelengths to our object spectrum

In [ ]:
# Add the wavelengths to the spectrum catalog
scat = Table(scat)
scat['wave'] = wmodel3

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.plot(scat['wave'],scat['flux'])
plt.xlabel('Wavelength (Angstroms)')
plt.ylabel('Flux')

# Continuum Normalization

Often we don't care about the absolute nature of the spectrum, but rather just the absorption lines.  Then it is useful to divide (normalize by) the continuum.

In [ ]:
def continuum(spec,bin=50,perc=60,norder=4):
    """ Derive the continuum of a spectrum."""
    nx = len(spec)
    x = np.arange(nx)
    # Loop over bins and find the maximum
    nbins = nx//bin
    xbin1 = np.zeros(nbins,float)
    ybin1 = np.zeros(nbins,float)
    for i in range(nbins):
        xbin1[i] = np.mean(x[i*bin:i*bin+bin])
        ybin1[i] = np.percentile(spec[i*bin:i*bin+bin],perc)
    # Fit polynomial to the binned values
    coef1 = np.polyfit(xbin1,ybin1,norder)
    cont1 = np.poly1d(coef1)(x)
    
    # Now remove large negative outliers and refit
    gdmask = np.zeros(nx,bool)
    gdmask[(spec/cont1)>0.8] = True
    xbin = np.zeros(nbins,float)
    ybin = np.zeros(nbins,float)
    for i in range(nbins):
        xbin[i] = np.mean(x[i*bin:i*bin+bin][gdmask[i*bin:i*bin+bin]])
        ybin[i] = np.percentile(spec[i*bin:i*bin+bin][gdmask[i*bin:i*bin+bin]],perc)
    # Fit polynomial to the binned values
    coef = np.polyfit(xbin,ybin,norder)
    cont = np.poly1d(coef)(x)
    
    return cont,coef

In [ ]:
cont,coef = continuum(scat['flux'],bin=20,norder=5)

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.plot(scat['flux'])
plt.plot(cont)

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.plot(scat['flux']/cont)
plt.ylim(0,1.2)

# Measuring Equivalent Widths

When doing abundances we want to equivalent widths of lines, basicaly the area in the line below the continuum (in Angstroms).

In [ ]:
flux = scat['flux']/cont
wave = scat['wave']

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.plot(flux)
plt.xlim(780,850)
plt.ylim(0,1.2)

In [ ]:
# It's easy to measure the equivalent width
# Sum(1-flux)*dw
dw = wave[801]-wave[800]
print(np.sum(1-flux[780:850])*dw)

In [ ]:
# To make this more accurate, let's fit a Gaussian to the line

In [ ]:
initpar = [-0.5,wave[811],2.0,1.0]
bnds = ([-1.0,initpar[1]-5,0.1,0.5], [0.0,initpar[1]+5,20,1.5])
pars,cov = gaussfit(wave[750:850],flux[750:850],initpar,bounds=bnds)
gmodel = gaussian(wave[750:850],*pars)
print(np.abs(pars[0])*pars[2]*np.sqrt(2*np.pi))

In [ ]:
fig = plt.figure(figsize=(10,6))
x = np.arange(len(flux))
plt.plot(x,flux)
plt.plot(x[750:850],gmodel)
plt.xlim(780,850)
plt.ylim(0,1.2)

In [ ]:
def voigt(x, height, cen, sigma, gamma, const=0.0, slp=0.0):
    """
    Return the Voigt line shape at x with Lorentzian component HWHM gamma
    and Gaussian sigma.

    """

    maxy = np.real(wofz((1j*gamma)/sigma/np.sqrt(2))) / sigma\
                                                           /np.sqrt(2*np.pi)
    return (height/maxy) * np.real(wofz(((x-cen) + 1j*gamma)/sigma/np.sqrt(2))) / sigma\
                                                           /np.sqrt(2*np.pi) + const + slp*(x-cen)

In [ ]:
def voigtfit(x,y,initpar=None,sigma=None,bounds=(-np.inf,np.inf)):
    """Fit a Voigt profile to data."""
    if initpar is None:
        initpar = [np.max(y),x[np.argmax(y)],1.0,1.0,np.median(y),0.0]
    func = voigt
    return curve_fit(func, x, y, p0=initpar, sigma=sigma, bounds=bounds)

In [ ]:
def voigtarea(pars):
    """ Compute area of Voigt profile"""
    sig = np.maximum(pars[2],pars[3])
    x = np.linspace(-20*sig,20*sig,1000)+pars[1]
    dx = x[1]-x[0]
    v = voigt(x,np.abs(pars[0]),pars[1],pars[2],pars[3])
    varea = np.sum(v*dx)
    return varea

In [ ]:
initpar = [-0.5,wave[811],2.0,1.0,1.0,0.0]
bnds = ([-1.0,initpar[1]-5,0.1,0.1,0.5,-0.5], [0.0,initpar[1]+5,20,20,1.5,0.5])
vpars,vcov = voigtfit(wave[780:850],flux[780:850],initpar,bounds=bnds)
print(vpars)
print(voigtarea(vpars))

In [ ]:
fig = plt.figure(figsize=(10,6))
x = np.arange(len(flux))
plt.plot(x,flux,label='Data')
plt.plot(x[780:850],vmodel,label='Voigt')
plt.xlim(780,850)
plt.ylim(0.2,1.3)
plt.legend()